<a href="https://colab.research.google.com/github/GurmitSeera/Test_repo/blob/main/SQL_Excercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

# Setup a password `postgres` for username `postgres`
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD 'postgres';"

# Setup a database with name `tfio_demo` to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS testdb;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE testdb;'
!pip install psycopg2

 * Starting PostgreSQL 10 database server
   ...done.
ALTER ROLE
ERROR:  database "testdb" is being accessed by other users
DETAIL:  There are 3 other sessions using the database.
ERROR:  database "testdb" already exists


In [ ]:
import pandas as pd
import psycopg2

In [ ]:
connection_string = {'host':'localhost',
                     'dbname':'testdb',
                     'user':'postgres',
                     'password':'postgres',
                     'port':5432}
connection = psycopg2.connect(**connection_string)

# Proble Statement 1

## Setup

In [ ]:
sql_query = """
create table salary(
  employee_id varchar(10),
  department_id varchar(10),
  salary double precision
);
"""
with connection.cursor() as cur:
  cur.execute(sql_query)
insert_query = """
insert into salary(employee_id, department_id, salary) 
values 
('1','1',100.0),
('2','1',200.0),
('3','1',300.0),
('4','2',400.0),
('5','2',600.0),
('6','2',100.0),
('7','3',500.0),
('8','3',500.0),
('9','4',1000.0)
;

"""
with connection.cursor() as cur:
  cur.execute(insert_query)

ProgrammingError: ignored

## Question 1
Show department_id, max_salary

In [ ]:
department_max_salary = """ Select department_id, max(salary) as max_salary from salary group by department_id;
"""
pd.read_sql_query(department_max_salary, connection)

## Question 2
Show department_id, employee_id, max_salary

In [ ]:
query = """select employee_id, department_id, salary from (Select employee_id, department_id, salary,
 rank () over (partition by department_id order by salary desc, employee_id) as s_rank
from salary) as A where s_rank = 1;
"""
pd.read_sql_query(query,connection)

## Question 3
Create employee_info table with columns employee_id, employee_name, city

In [ ]:

query_create_table2 = """ Create table employee_info(employee_id varchar(10),
employee_name varchar(64), city varchar(64)
);
"""
with connection.cursor() as cur:
  cur.execute(query_create_table2)

In [ ]:
insert_query1 = """ Insert into employee_info (employee_id, employee_name, city)
values
('1', 'Ramesh', 'Delhi'),
('2', 'Rahul', 'Gurugram'),
('3', 'Sandeep', 'Kolkata'),
('4', 'Deepanshu', 'Sonipat'),
('5', 'Sapna', 'Noida'),
('6', 'Varun', 'Noida'),
('7', 'Himanshu', 'Bhopal'),
('8', 'Ashwini', 'Jaipur'),
('9', 'Vipul','Panipat')
;
"""
with connection.cursor() as cur:
  cur.execute(insert_query1)


## Question 4
Show departmentid, employeeid, employeename, max salary

In [ ]:
query1 = """ select employee_id, department_id, employee_name, salary from (Select b.employee_id, b.department_id, b.salary, c.employee_name,
 rank () over (partition by b.department_id order by b.salary desc, b.employee_id) as s_rank
from salary b left join employee_info c on b.employee_id = c.employee_id) as A where s_rank = 1;
"""
pd.read_sql_query(query1,connection)

DatabaseError: ignored

## Question 5
Show employeeid, employeename, salary, salary %

In [ ]:
## get employeeid, employeename, salary, salary %
query2 = """ Select employee_id, employee_name, salary,
100.0*coalesce(salary,0)/sum(coalesce(salary,0)) over () as salary_percentage from (
Select s.employee_id, e.employee_name, s.salary
from salary s left join employee_info e on s.employee_id = e.employee_id) as A ;
"""
pd.read_sql_query(query2, connection)

## Question 6
Show employeeid, departmentid, salary where salary > 500.0

In [ ]:
## get employeeid, departmentid, salary where salary > 500.0
query3 = """ Select employee_id, department_id, salary from salary where salary > 500.0;
"""
pd.read_sql_query(query3,connection)

## Question 6
create new column in salary table having age of employee with (1,30), (2,30), (3,31), (4,31), (5,32) , (6,33), (7,32), (8,33), (9,31)

In [ ]:
Query4 = """ Begin transaction;
Alter table salary add age float(32) default 1.0 ;
Update salary set age = 30 where employee_id = '1';
Update salary set age = 30 where employee_id = '2';
Update salary set age = 31 where employee_id = '3';
Update salary set age = 31 where employee_id = '4';
Update salary set age = 32 where employee_id = '5';
Update salary set age = 33 where employee_id = '6';
Update salary set age = 32 where employee_id = '7';
Update salary set age = 33 where employee_id = '8';
Update salary set age = 31 where employee_id = '9';
END transaction;
"""
with connection.cursor() as cur:
  cur.execute(Query4)


## Question 7
Show age, average salary for each age

In [ ]:
query = """ Select age, avg(salary) as avg_salary_for_each_age from salary group by age;
"""
pd.read_sql_query(query, connection)

## Question 8
Show department id, age, average salary for each age and department

In [ ]:
query = """ Select department_id, age, avg(salary) as avg_salary_for_each_age from salary group by 1,2;
"""
pd.read_sql_query(query, connection)

## Question 9
Show department id, age, average salary for each age and department

## Question 10
update employee id 1 salary to 1000.0 from 100.0

In [ ]:
query = """ Update salary set salary = 100.0 where employee_id = '1';
"""
with connection.cursor() as cur:
  cur.execute(query)

In [ ]:
query = """Select * from salary;
"""
pd.read_sql_query(query, connection)

## Question 11
show table with employee id, employee name, city where salary > 500.0

In [ ]:
query = """ Select s.employee_id, e.employee_name, e.city, s.salary
from salary s left join employee_info e on s.employee_id = e.employee_id where s.salary > 500.0;
"""
pd.read_sql_query(query, connection)

## Problem Statement 2

In [ ]:
sql_query = """
create table infection(
  patient_id varchar(10),
  diagnosis_date date,
  infection_flag int
);
"""
with connection.cursor() as cur:
  cur.execute(sql_query)
insert_query = """
insert into infection(patient_id, diagnosis_date, infection_flag) 
values 
('a1','2020-07-01',0),
('a1','2020-07-02',0),
('a1','2020-07-03',1),
('a1','2020-07-04',0),
('a1','2020-07-05',0),
('b1','2020-07-04',1),
('b1','2020-07-03',1),
('b1','2020-07-02',0),
('c1','2020-07-01',1),
('c1','2020-07-02',0),
('c1','2020-07-04',1),
('c1','2020-07-07',0),
('c1','2020-07-03',1),
('c1','2020-07-09',0),
('c1','2020-07-14',1),
('c1','2020-07-15',0),
('c1','2020-07-16',0),
('c1','2020-07-17',1),
('c1','2020-07-18',0),
('c1','2020-07-19',1),
('c1','2020-07-11',0),
('d1','2020-07-21',0),
('d1','2020-07-11',1),
('d1','2020-07-22',0),
('d1','2020-07-12',0),
('d1','2020-07-14',1),
('d1','2020-07-16',0),
('d1','2020-07-18',1),
('d1','2020-07-23',0),
('d1','2020-07-24',1),
('d1','2020-07-25',0),
('d1','2020-07-26',1),
('d1','2020-07-27',0),
('d1','2020-07-28',1),
('d1','2020-07-30',0),
('d1','2020-07-31',1),
('e1','2020-07-20',0),
('e1','2020-07-21',1),
('e1','2020-07-22',0),
('e1','2020-07-23',1)
;

"""
with connection.cursor() as cur:
  cur.execute(insert_query)

## Question 1
Sort table on diagnosis_date ascending and create new column last_5_days_infection_flag

Logic - last_5_days_infection_flag will show if a patient had infection in last 5 days. 

In [ ]:
#query = """ SELECT  i.diagnosis_date,
	#	i.infection_flag,
#		sum(i2.infection_flag)
#		from Infection i 
#	cross apply
#		( SELECT * from Infection i1
#			where i1.diagnosis_date > dateadd(day, -5, i.diagnosis_date)
#			and i1.diagnosis_date < dateadd(day,1,i.diagnosis_date)) i2
#	group by i.diagnosis_date,
#		i.infection_flag order by i.diagnosis_date;
#"""
#pd.read_sql_query(query, connection)


# Question 2
Do in pandas

Sort table on diagnosis_date ascending and create new column last_5_days_infection_flag

Logic - last_5_days_infection_flag will show if a patient had infection in last 5 days. 


# Problem Statement 3

In [ ]:
Sql_query = """
CREATE TABLE Warehouses (
   Code INTEGER NOT NULL,
   Location VARCHAR(255) NOT NULL ,
   Capacity INTEGER NOT NULL,
   PRIMARY KEY (Code)
 );
CREATE TABLE Boxes (
    Code CHAR(4) NOT NULL,
    Contents VARCHAR(255) NOT NULL ,
    Value REAL NOT NULL ,
    Warehouse INTEGER NOT NULL,
    PRIMARY KEY (Code),
    FOREIGN KEY (Warehouse) REFERENCES Warehouses(Code)
 ) ;
"""
with connection.cursor() as cur:
  cur.execute(sql_query)
insert_query = """INSERT INTO Warehouses(Code,Location,Capacity)
                  values
                  (1,'Chicago',3),
                  (2,'Chicago',4),
                  (3,'New York',7),
                  (4,'Los Angeles',2),
                  (5,'San Francisco',8)
                  ;

                  INSERT INTO Boxes(Code,Contents,Value,Warehouse) 
                  VALUES
                  ('0MN7','Rocks',180,3),
                  ('4H8P','Rocks',250,1),
                  ('4RT3','Scissors',190,4),
                  ('7G3H','Rocks',200,1),
                  ('8JN6','Papers',75,1),
                  ('8Y6U','Papers',50,3),
                  ('9J6F','Papers',175,2),
                  ('LL08','Rocks',140,4),
                  ('P0H6','Scissors',125,1),
                  ('P2T6','Scissors',150,2),
                  ('TU55','Papers',90,5)
                  ;
"""

# Question 1
Select the code of each box, along with the name of the city the box is located in.

In [ ]:
query= """ SELECT Boxes.Code, Location
           FROM Warehouses 
           INNER JOIN Boxes
           ON Warehouses.Code = Boxes.Warehouse;
"""
pd.read_sql_query = (query, connection)